# Laboratorio 1

El siguiente notebook corresponde al desarrollo del laboratorio 1. Cada sección de código está totalmente comentada de manera que el funcionamiento sea entendible y eficiente para la persona el lector.

### Integrantes:

- Jesús David García Vargas - 2445

- Juan Felipe Santos Rodriguez - 2445

- Brayan de Jesús Cantillo Orozco - 2445

Nombre librería: funcionamiento o propósito de la librería.

Nombre librería: funcionamiento o propósito de la librería.

Nombre librería: funcionamiento o propósito de la librería.

In [1]:
#importamos las librerías necesarias

Comentarios...

In [2]:
#Leer el CSV preferiblemente con pandas

Comentarios...

In [3]:
class Row (object):
    title: str
    department: str
    city: str
    property_type: str
    latitude: float
    longitude: float
    surface_total: int
    surface_covered: int
    bedrooms: int
    bathrooms: int
    operation_type: str
    price: int

    # Constructor en donde inicializamos cada variable de las filas del dataset.
    def __init__ (self, title: str, department: str, city: str, property_type: str, 
    latitude: float, longitude: float, surface_total: int, surface_covered: int, 
    bedrooms: int, bathrooms: int, operation_type: str, price: int):
        title = self.title
        department = self.department
        city = self.city
        property_type = self.property_type
        latitude = self.latitude
        longitude = self.longitude
        surface_total = self.surface_total
        surface_covered = self.surface_covered
        bedrooms = self.bedrooms
        bathrooms = self.bathrooms
        operation_type = self.operation_type
        price = self.price

IndentationError: expected an indented block (2054504143.py, line 4)

In [4]:
class Node(object):
    metric: float
    data: Row
    left: "Node"
    right: "Node"
    height: int

    # Constructor para inicializar las propiedades del nodo.
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        self.height = 1
        self.metric = self.data.price / self.data.surface_total

    # función para generar mapa según latitud y longitud del nodo
    # usaremos alguna librería o la API de Google Maps
    def locate (self):
        pass

NameError: name 'Node' is not defined

In [5]:
#Definir la clase de Arbol...
# Método para mostrar (graphviz)
# Python code to delete a node in AVL tree
# Generic tree node class
 
class Tree(object):
    # Obtener el sucesor de un nodo dado.
    def __getSuccessor(self, node: Node):
        node = root.right
        while node is not None:
            node = node.left
        return node

    # Obtener la altura de un nodo dado.
    def __getHeight(self, root: Node):
        if not root:
            return 0
        return root.height
 
    # Obtener el factor de balanceo de un nodo dado.
    def getBalance(self, root: Node):
        if not root:
            return 0
        return self.__getHeight(root.right) - self.__getHeight(root.left)

    # Rotación simple izquierda.
    def __leftRotate(self, z):
        aux = node.right

        # Ejecutamos la rotación.
        node.right = aux.left;
        aux.left = node;

        # Actualizar alturas.
        node.height = 1 + max(self.__getHeight(node.left), self.__getHeight(node.right))
        aux.height = 1 + max(self.__getHeight(aux.left), self.__getHeight(aux.right))
 
        # Retornamos la nueva raíz.
        return aux
 
    # Rotación simple derecha.
    def __rightRotate(self, node):
        aux = node.left
 
        # Ejecutamos la rotación.
        node.left = aux.right;
        node.right = node;  
 
        # Actualizamos las alturas.
        node.height = 1 + max(self.__getHeight(node.left), self.__getHeight(node.right))
        aux.height = 1 + max(self.__getHeight(aux.left), self.__getHeight(aux.right))
 
        # Retornamos la nueva raíz.
        return aux

    # Insert node
    def insert(self, root: Node, data: Row) -> Node:
        # si no hay raíz, retornamos el nodo que sería la nueva raíz.
        if not root:
            return Node(data)
        # Si la métrica actual es menor a la del nodo actual, pasamos a la izquierda.
        elif (data.price / data.surface_total) < root.metric:
            root.left = self.insert(root.left, data)
        # Si la métrica actual es mayor a la del nodo actual, pasamos a la derecha.
        else:
            root.right = self.insert(root.right, data)
 
        # Actualizamos la altura del nodo.
        root.height = 1 + max(self.__getHeight(root.left), self.__getHeight(root.right))
 
        # Obtenemos el factor de balanceo.
        balance = self.__getBalance(root)
 
        # Si el nodo no está balanceado, revisamos cada uno de los 4 casos:
        # Rotación simple derecha.
        if balance < -1 and (data.price / data.surface_total) < root.left.metric:
            return self.__rightRotate(root)
 
        # Rotación simple izquierda.
        if balance > 1 and (data.price / data.surface_total) > root.right.metric:
            return self.__leftRotate(root)
 
        # Rotación doble izquierda derecha.
        if balance < -1 and (data.price / data.surface_total) > root.left.metric:
            root.left = self.__leftRotate(root.left)
            return self.__rightRotate(root)
 
        # Rotación doble derecha izquierda.
        if balance > 1 and (data.price / data.surface_total) < root.right.metric:
            root.right = self.__rightRotate(root.right)
            return self.__leftRotate(root)
 
        return root
 
    # Eliminar nodo
    def delete(self, root: Node, metric: float) -> Node:
        # Si no hay raíz, retornamos el puntero vacío.
        if not root:
            return root
        # Si la métrica es menor a la actual, nos vamos a la izquierda. 
        elif metric < root.metric:
            root.left = self.delete(root.left, metric)
        # Si la métrica es mayor a la actual nos vamos a la derecha.
        elif metric > root.metric:
            root.right = self.delete(root.right, metric)
        #Si la métrica es la misma y el nodo actual si existe.
        else:
            #Validamos los casos en los que el hijo izquierdo o derecho no existan.
            if root.left is None:
                temp = root.right
                root = None
                return temp
 
            elif root.right is None:
                temp = root.left
                root = None
                return temp
 
            # Obtenemos el sucesor del nodo actual.
            temp = self.__getSuccessor(root)
            root.metric = temp.metric
            root.right = self.delete(root.right, temp.metric)
 
        # Si el arbol solo tiene un nodo, lo retornamos.
        if root is None:
            return root
 
        # Actualizamos la altura del nodo.
        root.height = 1 + max(self.__getHeight(root.left), self.__getHeight(root.right))
 
        # Obtenemos el factor de balanceo.
        balance = self.__getBalance(root)
 
        # Si el arbol no está balanceado, aplicamos uno de los 4 casos:
        # Rotación simple derecha.
        if balance < -1 and self.__getBalance(root.left) <= 0:
            return self.__rightRotate(root)
 
        # Rotación simple izquierda.
        if balance > 1 and self.__getBalance(root.right) >= 0:
            return self.__leftRotate(root)
 
        # Rotación doble izquierda derecha.
        if balance < -1 and self.__getBalance(root.left) > 0:
            root.left = self.__leftRotate(root.left)
            return self.__rightRotate(root)
 
        # Rotación derecha izquierda.
        if balance > 1 and self.__getBalance(root.right) < 0:
            root.right = self.__rightRotate(root.right)
            return self.__leftRotate(root)
 
        return root
    
    # buscar un nodo utilizando la métrica dada
    # query será un diccionario del tipo { [campo]: [valor, comparador ("=" | ">" | "<" | ">=" | "<=")] }
    # Ejemplo: {"city": ["Barranquilla", "="], "bedrooms": [2, ">"]}
    def search(query) -> Node:
        #Validar que la entrada tenga el formato válido
        # * las claves solo pueden tener nombres de campos del dataset
        # * los valores tienen que ser de tipo lista y tener exactamente 2 elmentos
        # * los comparadores solo pueden ser: "=" | ">" | "<" | ">=" | "<="

        # Buscar el nodo y retornarlo.
        pass

    def traverse():
        # Mostrar el recorrido por niveles del árbol (de manera recursiva)
        pass

    # Graficar el arbol completo con la librería de graphviz
    def graph():
        pass
    


Comentarios...

In [6]:
class Utils(object):
    # obtener el nivel de un nodo.
    def level():
        pass

    # Encontrar el padre del nodo.
    def father (self):
        pass

    # Encontrar el abuelo del nodo.
    def grandfather (self):
        pass

    # Encontrar el tío del nodo.
    def uncle(self):
        pass

TypeError: __str__ returned non-string (type int)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5a6aa858-3dd7-43be-98c5-49e544ecec34' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>